In [1]:
import os
import sys

import numpy as np
import pandas as pd

import torch
import torch.nn as nn


sys.path.append('..')
sys.path.append('.')

from peft_modules import peft_utils
import esm_adapter

from prompt_tunning import PrefixTuning


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.hub.set_dir("./torch_hub")
torch.hub.set_dir("/home/zengs/zengs_data/torch_hub")

/home/zengs/data/anaconda3/envs/venv_pl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
INPUT_SEQ_SIZE = 10

# data
data = peft_utils.get_esm_example_data(INPUT_SEQ_SIZE)


# Load ESM-2 model
# esm2_t33_650M_UR50D
# esm2_t6_8M_UR50D
model, alphabet = esm_adapter.pretrained.esm2_t6_8M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()

batch_labels, batch_strs, batch_tokens = batch_converter(data)


Prefix Tuning (with prompt token = 10)

In [3]:

if True:
    prompt_model = PrefixTuning(model, prompt_len=10,
                                input_seq_len=INPUT_SEQ_SIZE,
                                prompt_layer_index=1,
                                device=DEVICE)

    model.prefix_module = prompt_model

    with torch.no_grad():
        results = model(batch_tokens, repr_layers=range(0, len(model.layers)+1))

    for i in range(0, len(model.layers)+1):
        print(f"Representations {i}: {results['representations'][i].shape}")

Representations 0: torch.Size([4, 12, 320])
Representations 1: torch.Size([4, 12, 320])
Representations 2: torch.Size([4, 22, 320])
Representations 3: torch.Size([4, 22, 320])
Representations 4: torch.Size([4, 22, 320])
Representations 5: torch.Size([4, 22, 320])
Representations 6: torch.Size([4, 12, 320])
